In [2]:
from dotenv import load_dotenv
import sys
import os


load_dotenv()
fin_app_path = os.path.abspath('..')
sys.path.append(fin_app_path)

In [3]:

from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

from datetime import datetime
from dateutil.relativedelta import relativedelta


from app.consts import l_llm, PINECONE_INDEX_NAME, EMBEDDING_MODEL, RAG_DOCS_FOL
from app.chains_rag import get_excerpt_chain, get_relevance_chain
from tqdm import tqdm
import json

# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate 
# from langchain_core.output_parsers import StrOutputParser


# from langchain_core.pydantic_v1 import BaseModel, Field
# from app.consts import default_prompt



In [3]:
LLM_s = l_llm['gpt-4o-mini']
LLM_l = l_llm['gpt-4o']

In [ ]:
## Iterating over all docs



{1: True, 2: False, 3: True}

In [ ]:
## Generating RELEVANCE
fol = '../../rag_docs'
rel_chain = get_relevance_chain()


d_rel_all = {}
l_file_names = os.listdir(fol)
for file_name in l_file_names:
    d_rel_all[file_name] = {}
    
    full_path = os.path.join(fol, file_name)
    print('LOADING PAGES FROM', file_name )
    loader = PyPDFLoader(full_path)
    pages = loader.load()
    print('PAGES LOADED FROM', file_name ,' Starting RELEVANCE CHECK')

    for page_no, page_content in tqdm(enumerate(pages)):
        d_rel_all[file_name][page_no] = rel_chain.invoke(page_content)
    # json.dump(d_rel_all, open('d_rel.json','w'))



LOADING PAGES FROM BHARTIARTL_AR_22.pdf
PAGES LOADED FROM BHARTIARTL_AR_22.pdf  Starting RELEVANCE CHECK


236it [03:47,  1.04it/s]


LOADING PAGES FROM BHARTIARTL_AR_23.pdf
PAGES LOADED FROM BHARTIARTL_AR_23.pdf  Starting RELEVANCE CHECK


249it [03:53,  1.07it/s]


LOADING PAGES FROM BHARTIARTL_AR_24.pdf
PAGES LOADED FROM BHARTIARTL_AR_24.pdf  Starting RELEVANCE CHECK


...

17it [05:01, 17.74s/it]


LOADING PAGES FROM BHARTIARTL_TRS_24Feb.pdf
PAGES LOADED FROM BHARTIARTL_TRS_24Feb.pdf  Starting RELEVANCE CHECK


10it [00:38,  3.81s/it]


LOADING PAGES FROM BHARTIARTL_TRS_24May.pdf
PAGES LOADED FROM BHARTIARTL_TRS_24May.pdf  Starting RELEVANCE CHECK


17it [01:23,  4.91s/it]


In [ ]:

# def get_rel_page_content(d_rel,file_name, l_pages):
#     rel_page_number = [int(k) for k,v in d_rel[file_name].items() if v ]
#     rel_page_content = {i: page_content for i,page_content in enumerate(l_pages) if i in rel_page_number}
#     return rel_page_content
 
get_rel_page_numbers = lambda d_rel,file_name: [int(k) for k,v in d_rel[file_name].items() if v ]


In [ ]:
## Generating EXCERPTS

exc_fil_path = "d_exc.json"
d_rel = json.load(open('d_rel.json','r'))
fol = RAG_DOCS_FOL
exc_chain = get_excerpt_chain()
get_rel_page_numbers = lambda d_rel,file_name: [int(k) for k,v in d_rel[file_name].items() if v ]


d_exc_all = json.load(open(exc_fil_path,'r'))
### json.dump(open(exc_fil_path,'w'))
l_file_names = os.listdir(fol)

for i_filname, file_name in enumerate(l_file_names):

    full_path = os.path.join(fol, file_name)
    print('LOADING PAGES FROM', file_name , i_filname,'out of ', len(l_file_names))
    loader = PyPDFLoader(full_path)
    pages = loader.load()
    
    ## Fetching relevant pages
    l_rel_page_num = get_rel_page_numbers(d_rel,file_name)

    print('PAGES LOADED FROM', file_name ,' Starting Extraction for ',len(pages),'pages, out of which relevant :', len(l_rel_page_num))


    for page_no, page_content in tqdm(enumerate(pages)):
        if page_no in l_rel_page_num:
            ## Check if it has already been done
            skip = False
            d_already_written = json.load(open(exc_fil_path,'r'))
            if file_name in d_already_written.keys():
                if str(page_no) in  d_already_written[file_name].keys():
                    skip = True
                # If file_name doesn't exist, create an empty dict 
            else:
                d_exc_all[file_name] = {}    
        
            if not skip:
                d_exc_all[file_name][page_no] = exc_chain.invoke(page_content)
                json.dump(d_exc_all, open(exc_fil_path,'w'))




LOADING PAGES FROM BHARTIARTL_AR_22.pdf 0 out of  16
PAGES LOADED FROM BHARTIARTL_AR_22.pdf  Starting Extraction for  236 pages, out of which relevant : 91


236it [13:19,  3.39s/it]


LOADING PAGES FROM BHARTIARTL_AR_23.pdf 1 out of  16
PAGES LOADED FROM BHARTIARTL_AR_23.pdf  Starting Extraction for  249 pages, out of which relevant : 99


249it [13:39,  3.29s/it]


LOADING PAGES FROM BHARTIARTL_AR_24.pdf 2 out of  16
PAGES LOADED FROM BHARTIARTL_AR_24.pdf  Starting Extraction for  261 pages, out of which relevant : 110


...

17it [01:29,  5.25s/it]


LOADING PAGES FROM BHARTIARTL_TRS_24Feb.pdf 14 out of  16
PAGES LOADED FROM BHARTIARTL_TRS_24Feb.pdf  Starting Extraction for  10 pages, out of which relevant : 8


10it [01:08,  6.86s/it]


LOADING PAGES FROM BHARTIARTL_TRS_24May.pdf 15 out of  16
PAGES LOADED FROM BHARTIARTL_TRS_24May.pdf  Starting Extraction for  17 pages, out of which relevant : 14


17it [00:00, 121.80it/s]


In [ ]:

def get_file_name_metadata(fil_name):
    
    name = fil_name.split('.')[0] # Removing extension
    l_split = name.split('_')
    ticker = l_split[0]+'.NS'
    doc_type = l_split[1]
    dur = l_split[2]

    months = ["Jan", "Feb", "Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    f_mon_str = lambda x: str(months.index(x)+1).rjust(2,"0")

    if len(dur)==5 and dur[:2].isnumeric() and dur[-3:] in months:
        start_date = "20"+dur[:2]+"-"+f_mon_str(dur[-3:])+'-01'
        end_end = (datetime.strptime(start_date, '%Y-%m-%d') + relativedelta(day=31) ).strftime('%Y-%m-%d')
    elif len(dur)==2 and dur.isnumeric():
        start_date = "20"+dur[:2]+"-01-01"
        end_end = "20"+dur[:2]+"-12-31"
    ret = {
        "ticker": ticker,
        "doc_type": doc_type,
        "start_date": start_date,
        "end_date"  : end_end
    }
    return ret


In [51]:
## Creating Pinecone documents from Extracted Passages

# fol = '../../rag_docs'
# l_file_names = os.listdir(fol)

d_exc_all = json.load(open(exc_fil_path))

l_documents = []

for file_name in d_exc_all.keys():
    try:
        file_metadata = get_file_name_metadata(file_name)
    except:
        print(file_name, 'is having problmes')
    doc_metadata = file_metadata.copy()
    for page_num in d_exc_all[file_name].keys():
        doc_metadata = file_metadata.copy()
        doc_metadata["page"] = page_num
        for exc in d_exc_all[file_name][page_num]:
            doc_metadata["tag"] = exc['tag']
            pc_doc = Document(page_content=exc['excerpt'], metadata=doc_metadata)
            l_documents.append(pc_doc)
l_documents

[Document(metadata={'ticker': 'BHARTIARTL.NS', 'doc_type': 'TRS', 'start_date': '2024-05-01', 'end_date': '2024-05-31', 'page': '2', 'tag': 'Financial Metrics'}, page_content='I want to start by thanking our investors for showing confidence in us through the recently concluded Bharti Hexacom IPO.'),
 Document(metadata={'ticker': 'BHARTIARTL.NS', 'doc_type': 'TRS', 'start_date': '2024-05-01', 'end_date': '2024-05-31', 'page': '2', 'tag': 'Sectoral Phenomenon'}, page_content='We were recognized globally for our ESG initiatives. We were awarded the most sustainable emerging market telco as per future investment initiative institutes top 250 emerging market ESG ranking.'),
 Document(metadata={'ticker': 'BHARTIARTL.NS', 'doc_type': 'TRS', 'start_date': '2024-05-01', 'end_date': '2024-05-31', 'page': '2', 'tag': 'Financial Metrics'}, page_content='Our consolidated revenue came in at just under 150,000 Crores and was impacted by currency devaluation in Africa.'),
 Document(metadata={'ticker':

In [58]:
## Uploading docs to Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone

from langchain_pinecone import PineconeVectorStore
from app.consts import PINECONE_INDEX_NAME, EMBEDDING_MODEL


In [ ]:

vectorstore = PineconeVectorStore.from_documents(
    l_documents,
    index_name=PINECONE_INDEX_NAME,
    embedding=EMBEDDING_MODEL
    )


In [ ]:
## Querying for trial
pc = Pinecone()
index = pc.Index(name=PINECONE_INDEX_NAME,)
q = "Which product lines have seen the biggest improvements in profit margins"

res = index.query(
   query={
        "inputs": {"text": q}, 
        "top_k": 5
    },
    filter={"doc_type": {"$eq": "TRS"}},
    include_metadata=True # Include metadata in the response.
)



PineconeException: UNKNOWN:Error received from peer  {created_time:"2025-02-25T04:11:35.1162819+00:00", grpc_status:3, grpc_message:"Either \'vector\' or \'ID\' must be provided"}

In [66]:
vectorstore = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=EMBEDDING_MODEL)

# For retrival, only metadata_tags can be used. Only selecting those from the classification
filter = { 
        "$and": 
        [
            {"doc_type"    : {"$in":["TRS","AR"]}},
            {"tag"         : {"$eq":'Operational Metrics'} }
      ] 
     }
 
retriver = vectorstore.as_retriever(
    search_kwargs= {
        "filter": filter,
        "k": 5})
docs = retriver.invoke(q)
docs

[Document(id='887b176f-e399-4e25-9dc5-f585e509d91b', metadata={'doc_type': 'TRS', 'end_date': '2023-08-31', 'page': '4', 'start_date': '2023-08-01', 'tag': 'Operational Metrics', 'ticker': 'BHARTIARTL.NS'}, page_content='with the increased fiber capacity our focus is now on sales force productivity'),
 Document(id='7f0ed6c1-71f1-40d4-96d0-58d8d7fb4a62', metadata={'doc_type': 'AR', 'end_date': '2024-12-31', 'page': '74', 'start_date': '2024-01-01', 'tag': 'Operational Metrics', 'ticker': 'BHARTIARTL.NS'}, page_content='The mobile segment achieved healthy operating performance, driven by continued RMS gains, underlying operating leverage and relentless focus on improving cost efficiencies.'),
 Document(id='bec9e177-cf93-423e-8f48-f628c8fec79d', metadata={'doc_type': 'TRS', 'end_date': '2024-08-31', 'page': '6', 'start_date': '2024-08-01', 'tag': 'Operational Metrics', 'ticker': 'BHARTIARTL.NS'}, page_content='a large part of that really does flow through into the bottom-line so to that e

In [19]:
s = pages[0].page_content
print(s)

bharti� 
July 21, 2022 
National Stock Exchange of India Limited 
Exchange Plaza, C-1 Block G 
Bandra Kurla Complex, Bandra (E), 
Mumbai-400051, India 
Symbol: BHARTIARTU AIRTELPP 
BSE Limited 
Phiroze Jeejeebhoy Towers 
Dalal Street, 
Mumbai-400001, India 
Scrip Code: 532454/ 890157 
Sub: Notice of 27th Annual General Meeting, Integrated Report and Annual Financial Statements 
2021-22 and Record date for the payment of dividend 
Dear Sir/ Madam, 
Pursuant to Regulations 30, 34, 42 and 44 and other applicable provisions of the SEBI (Listing Obligations 
and Disclosure Requirements) Regulations, 2015 ('Listing Regulations'), we hereby submit as under: 
1. The 27th Annual General Meeting (AGM) of the Company will be held on Friday, August 12, 2022 at
11 :00 A.M. (1ST) through video conferencing/ Other Audio Visual Means in accordance with relevant
circulars issued by the Ministry of Corporate Affairs and the Securities and Exchange Board of India
('Circul�rs').
2. In terms of the said Ci

In [82]:
from langchain.prompts import ChatPromptTemplate 
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from app.consts import l_llm
from typing import Literal, List
from pydantic import BaseModel, Field
from app.consts import create_prompt

def f_get_excerpt_chain():
    class Excerpt(BaseModel):
        """ 
        An excerpt with it's tag
        """ 
        excerpt  : str = Field(description = "A small passage from a company's document")
        tag      : Literal["Operational Metrics",  "Financial Metrics",  "Key Personell",  "Product Strategy",  "Expansion" , "Competition",  "Sectoral Phenomenon",  "Regulatory", "Macro Factors",  "Organizational Details",  "MnA", "Alliances"] = Field(description = "Tag of the passage")

    class ListExcerpts(BaseModel):
        """List of excerpt and  tags""" 
        l_excerpt : List[Excerpt] = Field(description="List of excerpt and  tags")
         

    excerpt_prompt = """
    You are a business analyst extracting relevant information from a passage of a company's annual report.
    Note that given the parsing process of the document, some unneccessary newline or whitespaces may have been added in the passage.
    You have been provided with a list of important tags below along with their description. 
    Go through the contents of the input provided to you, and  find excerpts which talks about these one of these tags: ["Operational Metrics",  "Financial Metrics",  "Key Personell",  "Product Strategy",  "Expansion" , "Competition",  "Sectoral Phenomenon",  "Regulatory", "Macro Factors",  "Organizational Details",  "MnA", "Alliances"] 
    Your goal is to to extract all relevant excerpts. Each excerpt can range in length from a phrase to a sentence or at max 2-3 sentences, and should be as short as possible while making semantic sense.
    For each excerpt identify the most relevant tag. Only use the tags from the below list.
    Your response should the different excerpts with their related tag, formatted as a list of tuples
    Sample response:
    [
        {{"excerpt": "The Company has purchased additional 4.7% equity interest in org X from org Y", "tag": "MnA"}},
        {{"excerpt": "In partnership with org X, the company has conducted India's first 5G trial in the 700 MHz band.", "tag": "Alliances"}},
        {{"excerpt": "Company's X division launched feature Y, a digital platform for customers to be able to do Z", "tag": "Product Strategy"}},
        {{"excerpt": "The Company had 208.4 million data customers at the end of March 31, 2022, of which 200.8 million were segment X customers.", "tag": "Operational Metrics"}},
        {{"excerpt": "Consequently, EBIT for the year was 248,531 Mn, increasing by 49.6% and resulting in a margin of 21.3% vis-a-vis 16.5% in the previous year.", "tag": "Financial Metrics"}}
    ]
    If the page talks about none of these tags, respond with an empty list [].

    List of Tags to choose from
    - Operational Metrics: Any excerpt that talks about the companies operational metrics for the business, eg. Average Revenue for User, Churn Rate, Net Promoter Score, Resoliton Time etc. Basically anything that indicates how well the company is serving its customers.
    - Financial Metrics: Any excerpt talking about the companies financia; metrics for the business, eg.any Revenue, Costs, Profits, Debts, Valuation, Share Price etc.
    - Key Personell: Any excerpt talking about any executive, management or key personell like a CxO,MD, board of directors etc. - any announcement regarding a change in the personell, modification in their role. or their achievement. 
    - Product Strategy: If the excerpt talks about it's companies portfolio of products or offerings. If any particular product is doing very well or poorly, or if they have changed the product portfolio - adding or removing some items, or changing their focus areas.
    - Expansion: If the excerpt is talking about expansion plans of the organization. This could be by capital expenditure, vertical growth by entering into some other part of the value chain, horizontal growth by targetting a different product line or customer segment, or entering a new business line. This could be an organic growth with internal team working on the new initiative, or inorganic growth via by Mergers or Acquisitions.
    - Competition: If the excerpt talks about it's competitor or the compeitive landscape for its industry.
    - Sectoral Phenomenon: If the excerpt talks about any wholesale changes in consumer preferences; price changes in key ingredients like commodities, energy, labour ; technological advancements 
    - Regulatory: If the excerpt talks about the regulatory landscape, or if the government or any other governing body's policy has impacted the company's business in any way.
    - Macro Factors: If the excerpt talks about any macro phenomena that impacts the business - like global politics, trade treaties, war etc. Also if the excerpt talks about country's economics and populations well being 
    - Organizational Details: If the excerpt talks about the organization structure, or about any parent or subsidiary organization.
    - MnA: If the excerpt talks about any mergers or acquisisions that the company has undertaken.
    - Alliances: If the company has entered into a strategic alliance or partnership with any other organization
    Ensure that the each of tag is one of these: ["Operational Metrics",  "Financial Metrics",  "Key Personell",  "Product Strategy",  "Expansion" , "Competition",  "Sectoral Phenomenon",  "Regulatory", "Macro Factors",  "Organizational Details",  "MnA", "Alliances"] = Field(description = "Tag of the passage")
    Do NOT use any other tag
    """.strip()
    get_l_excerpts =  RunnableLambda(lambda x: x.model_dump()['l_excerpt'])

    llm_l_st = LLM_l.with_structured_output(ListExcerpts)
    chain = create_prompt(excerpt_prompt) | llm_l_st | get_l_excerpts

    return chain





In [17]:
from typing import Literal, List
from pydantic import BaseModel, Field
from app.consts import create_prompt
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

def f_get_relevance_chain():
    class Relevance(BaseModel):
        """ Whether a page is relevant or not """ 
        rel : Literal["RELEVANT",  "IRRELEVANT"] = Field(description = "Whether the page-content is relevant or not")
        
    relevance_prompt = """
    You are a equity research analyst. 
    You will be provided a passage from a company's filing report. 
    Your task is to check the passage is relevant in better understanding its business or not.
    If the passage is relevant if it talks about anything regarding the company's product portfolio, operational metrics or the macro factors, sectoral phenomena, competition, or company's plan for expansion, mergers, etc.
    It is irrelevant if the passage seems is filled with legal jargon or are highly technical details of numbers or numerical calculations, or seems completely unrelated to the business.
    Your response has to a single word - "RELEVANT" or "IRRELEVANT"
    """
    chain = create_prompt(relevance_prompt) | LLM_s.with_structured_output(Relevance) | RunnableLambda(lambda x: x.rel == "RELEVANT")
    return chain


In [20]:
chain = f_get_relevance_chain()
res = chain.invoke(pages[0].page_content)
res

False

In [83]:
excerpt_chain = f_get_excerpt_chain()
res = excerpt_chain.invoke(s)
res


[{'excerpt': 'Mr. Gopal Vittal has been elected to the Board of GSMA for the 2019-20 term.',
  'tag': 'Key Personell'},
 {'excerpt': 'industry has witnessed heavy turbulence in form of ARPU contraction and industry consolidation.',
  'tag': 'Sectoral Phenomenon'},
 {'excerpt': 'Additionally, the consumption metrics per subscriber are among the highest globally and continues to increase even further.',
  'tag': 'Operational Metrics'},
 {'excerpt': 'Despite a challenging industry environment and COVID-19 related disruptions over the last few years, Airtel delivered strong improving results year after year, through a competitive growth and revenue market share expansion in each of the business segment.',
  'tag': 'Financial Metrics'},
 {'excerpt': 'Cornerstone of this performance has been the undivided focus on customer delight through an exceptional experience backed by brilliant network, aspirational brand and omni-channel presence.',
  'tag': 'Operational Metrics'},
 {'excerpt': 'focus

In [ ]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings


doc = Document( page_content=res[0]['excerpt'],
    metadata={"doc": "BHARTIARTL_AR_22.pdf", "page" : 24, "tag" : res[0]['tag']}
)
doc

vectorstore = PineconeVectorStore.from_documents(
    [doc],
    index_name='fina-app-feb25',
    embedding= OpenAIEmbeddings()
)

